<div style="font-family: 'JetBrains Mono', monospace; font-size: 14px; color: #e2dbdbff; line-height: 1.6;">

## 📊 Exploratory Data Analysis (EDA)

### ✅ Principales insights:

- **25.29%** de los comercios tienen presencia web.
- **Tandil** y **Olavarria** lideran en digitalización.
- **Instagram 19.2%** y **Facebook 18.11** concentran la presencia en RRSS
- Los comercios con sitio web tienen **mayor cantidad de reviews** en todas las ciudades.
- El puntaje promedio se mantiene **alto y constante (4.0 - 4.4)**.
- Categorías más comunes:
  - 
  - 
  - 
  - 

---

</div>

In [1]:
import pandas as pd
import os
import numpy as np
from glob import glob
import glob
import matplotlib.pyplot as plt
import seaborn as sns

# Configura pandas para mostrar todas las columnas del DataFrame al mostrarlo, sin recorte visual.
pd.set_option('display.max_columns', None) 

In [8]:
df = pd.read_csv('../Python_es/CleanData/Regional.csv')

In [9]:
df.shape

(5859, 16)

In [19]:
df.size

93744

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5859 entries, 0 to 5858
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         5859 non-null   object 
 1   categoryName  5859 non-null   object 
 2   website       5859 non-null   object 
 3   street        5859 non-null   object 
 4   phone         5859 non-null   object 
 5   city          5859 non-null   object 
 6   state         5859 non-null   object 
 7   location/lat  5859 non-null   float64
 8   location/lng  5859 non-null   float64
 9   totalScore    5859 non-null   float64
 10  rank          5859 non-null   int64  
 11  imageUrl      5859 non-null   object 
 12  imagesCount   5859 non-null   float64
 13  reviewsCount  5859 non-null   int64  
 14  scrapedAt     5859 non-null   object 
 15  searchString  5859 non-null   object 
dtypes: float64(4), int64(2), object(10)
memory usage: 732.5+ KB


In [6]:
df.dtypes

title            object
categoryName     object
website          object
street           object
phone            object
city             object
state            object
location/lat    float64
location/lng    float64
totalScore      float64
rank              int64
imageUrl         object
imagesCount     float64
reviewsCount      int64
scrapedAt        object
searchString     object
dtype: object

In [15]:
# Convertir fecha de scraping manteniendo formato datetime
# Esto permite análisis temporales futuros (ej: comparar con scraping en 6 meses)
df['scrapedAt'] = pd.to_datetime(
    df['scrapedAt'], 
    errors='coerce',  # Maneja fechas inválidas sin romper el código
    utc=True          # Estandariza zona horaria para comparaciones
).dt.floor('s').dt.tz_localize(None)  # Elimina timezone para visualización limpia
# Resultado: 2025-06-15 22:56:55
df['scrapedAt']

0      2025-06-15 22:56:55
1      2025-06-15 22:56:54
2      2025-06-15 22:56:57
3      2025-06-15 22:56:57
4      2025-06-15 22:56:58
               ...        
5854   2025-07-30 15:45:05
5855   2025-07-30 15:45:19
5856   2025-07-30 15:45:19
5857   2025-07-30 15:45:06
5858   2025-07-30 15:45:04
Name: scrapedAt, Length: 5859, dtype: datetime64[ns]

In [20]:
df.head(10)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Dti - Azul,Soporte Y Servicios Informáticos,Incomplete,Av. Juan Domingo Perón 525,+54 9 2281 42-7066,Azul,Provincia de Buenos Aires,-36.782006,-59.865497,0.0,22,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,0,2025-06-15 22:56:55,Tecnologia
1,Tecno Computacion,Tienda De Móviles,http://www.tecnocomputacionazul.com/,Lamadrid 33,+54 9 2281 65-7938,Azul,Provincia de Buenos Aires,-36.764914,-59.862404,4.6,3,https://lh3.googleusercontent.com/p/AF1QipP9Gg...,12.0,7,2025-06-15 22:56:54,Tecnologia
2,Ingeniero Marcelo Edgardo Cornec,Ingeniero,Incomplete,Av. Juan Domingo Perón 622,+54 9 2281 42-6984,Azul,Provincia de Buenos Aires,-36.783278,-59.863594,5.0,20,https://lh3.googleusercontent.com/p/AF1QipN3mw...,13.0,3,2025-06-15 22:56:57,Comercio
3,La Esquina,Kiosco,Incomplete,"7300, Gral. Paz 1500",+54 9 2281 36-0736,Azul,Provincia de Buenos Aires,-36.798090,-59.853525,3.7,19,https://lh3.googleusercontent.com/p/AF1QipP1o8...,1.0,3,2025-06-15 22:56:57,Comercio
4,Vidrieria 25 De Mayo De Jose Canalicchio,Comercio,Incomplete,Guaminí 610,+54 9 2281 53-7036,Azul,Provincia de Buenos Aires,-36.785552,-59.867145,4.2,40,https://lh3.googleusercontent.com/p/AF1QipM1o-...,5.0,9,2025-06-15 22:56:58,Comercio
5,El Almacén,Tienda De Ropa,Incomplete,Av. Mitre 901,+54 9 2281 43-6001,Azul,Provincia de Buenos Aires,-36.782064,-59.853528,4.1,37,https://lh3.googleusercontent.com/gps-cs-s/AC9...,2.0,15,2025-06-15 22:56:58,Comercio
6,Tus Sorrentinos,Comercio,Incomplete,España 418,+54 9 2281 56-4122,Azul,Provincia de Buenos Aires,-36.779354,-59.853713,0.0,36,https://lh3.googleusercontent.com/p/AF1QipPp_b...,22.0,0,2025-06-15 22:56:58,Comercio
7,Venden Y Permutas,Tienda General,Incomplete,Gral. Rivas 787,Incomplete,Azul,Provincia de Buenos Aires,-36.788767,-59.865031,0.0,34,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,0,2025-06-15 22:56:58,Comercio
8,Minimercado La Esquina De Alejandro Y Silvina,Comercio,Incomplete,España 1699,Incomplete,Azul,Provincia de Buenos Aires,-36.796488,-59.873790,4.3,29,https://lh3.googleusercontent.com/gps-cs-s/AC9...,20.0,22,2025-06-15 22:56:58,Comercio
9,Kiosco Pasaje Del Parque,Comercio,Incomplete,Pje. 25 de Abril,+54 9 2281 52-6194,Azul,Provincia de Buenos Aires,-36.781831,-59.875032,5.0,23,https://lh3.googleusercontent.com/gps-cs-s/AC9...,6.0,1,2025-06-15 22:56:58,Comercio


In [22]:
df.tail(10)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
5849,Mundo Peugeot - Repuestos Y Accesorios,Tienda De Repuestos Para Automóviles,Incomplete,Gral. Rodríguez 1655,+54 9 249 402-4966,Tandil,Provincia de Buenos Aires,-37.313746,-59.143988,4.5,75,https://lh3.googleusercontent.com/p/AF1QipPA8V...,11.0,86,2025-07-30 15:45:17,Repuestos
5850,Riders Repuestos,Tienda De Repuestos Para Automóviles,Incomplete,Gral. Rodríguez 1495,Incomplete,Tandil,Provincia de Buenos Aires,-37.316190,-59.142774,0.0,70,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,0,2025-07-30 15:45:17,Repuestos
5851,Santamarina Repuestos,Tienda De Repuestos Para Automóviles,Incomplete,Av. Santamarina 36,Incomplete,Tandil,Provincia de Buenos Aires,-37.331030,-59.126801,0.0,69,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,0,2025-07-30 15:45:17,Repuestos
5852,Repuestos Buzon,Tienda De Repuestos Para Automóviles,Incomplete,Av. Balbín 1073,+54 9 249 448-7106,Tandil,Provincia de Buenos Aires,-37.313849,-59.122081,4.9,66,https://lh3.googleusercontent.com/gps-proxy/AL...,2.0,49,2025-07-30 15:45:17,Repuestos
5853,Kain Repuestos Gas Y Agua,Tienda De Artículos Relacionados Con El Gas,Incomplete,Leandro Alem 1217,+54 9 249 425-1165,Tandil,Provincia de Buenos Aires,-37.317811,-59.138120,4.8,68,https://lh3.googleusercontent.com/p/AF1QipNt9j...,65.0,5,2025-07-30 15:45:17,Repuestos
5854,Arkansas Muebles,Tienda De Muebles,http://arkansastandil.mitiendanube.com/,Av. Marconi 1498,+54 9 249 464-3311,Tandil,Provincia de Buenos Aires,-37.317572,-59.125688,5.0,79,https://lh3.googleusercontent.com/gps-proxy/AL...,3.0,1,2025-07-30 15:45:05,Electrodomesticos
5855,Pilchas Mb,Tienda De Ropa De Hombre,Incomplete,San Martín 915,+54 9 223 555-9393,Tandil,Provincia de Buenos Aires,-37.324699,-59.132030,5.0,78,https://lh3.googleusercontent.com/gps-cs-s/AC9...,6.0,9,2025-07-30 15:45:19,Ropa
5856,Rodriggi,Tienda De Ropa De Hombre,Incomplete,9 de Julio 445,+54 9 249 457-5285,Tandil,Provincia de Buenos Aires,-37.327796,-59.133822,4.9,80,https://lh3.googleusercontent.com/p/AF1QipPNSw...,6.0,23,2025-07-30 15:45:19,Ropa
5857,Lg Service,Servicio De Reparación De Electrodomésticos,Incomplete,Saavedra 778,+54 9 249 438-8237,Tandil,Provincia de Buenos Aires,-37.318255,-59.121669,4.7,84,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,3,2025-07-30 15:45:06,Electrodomesticos
5858,Friototal,Distribuidor De Acero,http://www.friototal.com.ar/,Av. del Valle 1258,+54 9 249 469-8239,Tandil,Provincia de Buenos Aires,-37.309298,-59.134556,3.7,60,https://lh3.googleusercontent.com/gps-cs-s/AC9...,17.0,3,2025-07-30 15:45:04,Electrodomesticos


In [23]:
df.sample(10)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
2665,Herrería Arias,Empresa Constructora,Incomplete,República del Líbano 9,Incomplete,Olavarría,Provincia de Buenos Aires,-36.889159,-60.311595,0.0,50,Incomplete,0.0,0,2025-07-30 14:42:48,Construccion
24,Maxikioscos Gonzalez,Librería,Incomplete,San Martín 532,+54 9 2281 42-4066,Azul,Provincia de Buenos Aires,-36.778935,-59.862150,4.5,63,https://lh3.googleusercontent.com/p/AF1QipN9Fh...,4.0,4,2025-06-15 22:57:01,Comercio
4775,Duque Indumentaria,Tienda De Ropa,Incomplete,José Cabral 2266,+54 9 249 427-1494,Tandil,Provincia de Buenos Aires,-37.298055,-59.168963,5.0,21,https://lh3.googleusercontent.com/p/AF1QipOPp3...,5.0,1,2025-05-29 00:17:25,Ropa
3347,Taller Poffer,Taller De Maquinaria,Incomplete,Incomplete,+54 9 249 456-3552,Rauch,Provincia de Buenos Aires,-36.774672,-59.090198,0.0,14,https://lh3.googleusercontent.com/p/AF1QipPzD1...,1.0,0,2025-06-15 22:40:49,Maquinaria
3396,Club Atlético Y Social San Lorenzo De Rauch,Club Deportivo,https://www.facebook.com/sanlorenzo.rauch/,Incomplete,Incomplete,Rauch,Provincia de Buenos Aires,-36.789986,-59.087044,4.7,3,https://lh3.googleusercontent.com/gps-cs-s/AC9...,9.0,26,2025-07-30 15:09:16,Deportivo
553,Vete Predio,Club Deportivo,Incomplete,1 de Mayo 2742,Incomplete,Azul,Provincia de Buenos Aires,-36.803424,-59.901643,0.0,15,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,0,2025-07-30 14:51:30,Deportivo
4123,"Cicconi Equipos Multifuncionales, Fotocopiador...",Servicio De Reparación De Impresoras,http://www.cicconi.com.ar/,11 de Septiembre 1057,+54 9 249 442-6639,Tandil,Provincia de Buenos Aires,-37.316311,-59.127947,4.6,18,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,24,2025-05-28 23:36:12,Electronica
5528,Materiales Menchón,Tienda De Materiales Para La Construcción,https://www.guia-dorada.com.ar/guia/aviso-corr...,Av. Balbín 1168,+54 9 249 458-3577,Tandil,Provincia de Buenos Aires,-37.312843,-59.123088,4.6,30,https://lh3.googleusercontent.com/p/AF1QipPMmU...,55.0,57,2025-07-30 15:43:11,Construccion
1869,Tiempo De Aventura Sa,Tienda De Ropa,http://www.tiempodeaventura.store/,"Esquina Moreno, Gral. Paz 2911",+54 9 2284 38-6262,Olavarría,Provincia de Buenos Aires,-36.895170,-60.322147,4.8,50,https://lh3.googleusercontent.com/gps-cs-s/AC9...,34.0,55,2025-06-11 18:13:57,Comercio
2594,Bazar Zeta,Bazar,Incomplete,Incomplete,Incomplete,Olavarría,Provincia de Buenos Aires,-36.890036,-60.318933,0.0,12,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,0,2025-07-30 14:42:43,Bazar
